In [ ]:
#To use, you should have the wikipedia python package installed.
#This wrapper will use the Wikipedia API to conduct searches and fetch page summaries. 
#By default, it will return the page summaries of the top-k results.
#It limits the Document content by doc_content_chars_max.
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper


In [ ]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun,DuckDuckGoSearchRun
#this is basically interacting the tools itself with outside the world with 
#the information present in arxiv or wikipedia

In [35]:
search=DuckDuckGoSearchRun(name="Search")#anything that search from internet

In [36]:
search.name

'Search'

In [4]:
from langchain.agents import initialize_agent,AgentType

In [ ]:
wikipediawrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=wikipediawrapper)

In [6]:
wiki.name

'wikipedia'

In [7]:
arxivwrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=arxivwrapper)

In [8]:
arxiv.name

'arxiv'

In [39]:
#creating our own custom tools

In [10]:
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader(web_paths=("https://www.britannica.com/biography/A-P-J-Abdul-Kalam",),)
docs=loader.load()
docs

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://www.britannica.com/biography/A-P-J-Abdul-Kalam', 'title': 'A.P.J. Abdul Kalam | Biography, History, Career, Books, Awards, & Facts | Britannica', 'description': 'A.P.J. Abdul Kalam, the esteemed scientist and former president of India, left an enduring legacy through his contributions to space research and inspired millions with his vision for a prosperous country.', 'language': 'en'}, page_content="\n\n\n\n\n\n\n\n\n\n\n\n\nA.P.J. Abdul Kalam | Biography, History, Career, Books, Awards, & Facts | Britannica\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch Britannica\n\n\n\n\n\nClick here to search\n\n\n\n\n\n\n\n\n\n\n\nSearch Britannica\n\n\n\n\n\nClick here to search\n\n\n\n\n\n\n\n   Subscribe\n\n\n\n   Subscribe\n\n\nLogin\n\nhttps://premium.britannica.com/premium-membership/?utm_source=premium&utm_medium=nav-login-box&utm_campaign=evergreen\n\n\n\n\n\n\n\n  Subscribe \xa0Now\n\n\n\n\nHome\nHisto

In [11]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFaceEmbeddings

In [13]:
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

In [14]:
groq_api_key=os.getenv("GROQ_API_KEY")

In [15]:
embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\user\OneDrive\Desktop\langchain\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [16]:
model=ChatGroq(model="gemma2-9b-it",api_key=groq_api_key)

In [17]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool

In [18]:
documents=RecursiveCharacterTextSplitter(chunk_size=400,chunk_overlap=100).split_documents(docs)
vector_db=FAISS.from_documents(documents,embedding=embedding)
db=vector_db.as_retriever()
db

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001DE79923740>, search_kwargs={})

In [ ]:
#we are able to create a custom tool but it is vector form for that
#in order use it as a tool we convert db into a tool fot we use create_retriever_tool


In [19]:
retriver_tool=create_retriever_tool(db,"abdul kalam",description="search information about dr APJ apdul kalam sir")

In [20]:
tools=[wiki,arxiv,retriver_tool]

In [21]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\user\\OneDrive\\Desktop\\langchain\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='abdul kalam', description='search information about dr APJ apdul kalam sir', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001DE45500C20>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object a

In [22]:
#there are different different prompts  are already created in the langchain 
# so we can easily able to pull it to our code

In [23]:
#[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
#MessagesPlaceholder(variable_name='chat_history', optional=True),
#HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
#MessagesPlaceholder(variable_name='agent_scratchpad')]
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")

In [24]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [25]:
from langchain.agents import create_openai_tools_agent

In [40]:
#for create_tools
agent=create_openai_tools_agent(llm=model,tools=tools,prompt=prompt)

In [28]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [29]:
# Create an agent executor by passing in the agent and tools
from langchain.agents import AgentExecutor

In [30]:
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [31]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [38]:
output=agent_executor.invoke({"input":"what is machine learning"})
output



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks witMachine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without being explicitly programmed. 

It involves training algorithms on large datasets to identify patterns and relationships, enabling them to make predictions or decisions on new, unseen data. 





> Finished chain.


{'input': 'what is machine learning',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without being explicitly programmed. \n\nIt involves training algorithms on large datasets to identify patterns and relationships, enabling them to make predictions or decisions on new, unseen data. \n\n\n\n'}

In [41]:
#!pip install --upgrade nbformat
#!pip install --upgrade nbconvert